In [40]:
import pandas as pd
import json
from pathlib import Path

raw_path = Path("../data/raw/corn_exports_2025.json")

# Open raw data
file = open(raw_path, "r")
data = json.load(file)

# Store data into dataframe
exports = pd.DataFrame(data)

# Convert dates
exports["weekEndingDate"] = pd.to_datetime(exports["weekEndingDate"])
exports.rename(columns={
    "weekEndingDate": "date",
    "weeklyExports": "weekly_exports_mt"
}, inplace=True)

# The data is already sorted by date, and there are no empty values
weekly_totals = exports.groupby("date")["weekly_exports_mt"].sum().reset_index()
weekly_totals.head()

,date,weekly_exports_mt
0,2025-09-04,687326
1,2025-09-11,1555901
2,2025-09-18,1315867
3,2025-09-25,1429180
4,2025-10-02,1830620


In [41]:
# Weekly percent change
weekly_totals["weekly_percent_change"] = weekly_totals["weekly_exports_mt"].pct_change()

# Moving average (4 week)
weekly_totals["moving_average_4w"] = weekly_totals["weekly_exports_mt"].rolling(window=4).mean()

# Standard deviation (4 week)
weekly_totals["std_4w"] = weekly_totals["weekly_exports_mt"].rolling(window=4).std()
weekly_totals

,date,weekly_exports_mt,weekly_percent_change,moving_average_4w,std_4w
0,2025-09-04,687326,NaN,NaN,NaN
1,2025-09-11,1555901,1.263702,NaN,NaN
2,2025-09-18,1315867,-0.154273,NaN,NaN
3,2025-09-25,1429180,0.086113,1247068.50,385826.822723
4,2025-10-02,1830620,0.280888,1532892.00,221380.069589
5,2025-10-09,1122480,-0.386831,1424536.75,298871.522278
6,2025-10-16,1410130,0.256263,1448102.50,291061.060074
7,2025-10-23,1332502,-0.055050,1423933.00,297107.375510
8,2025-10-30,1540714,0.156256,1351456.50,175167.055056
9,2025-11-06,1528644,-0.007834,1452997.50,99621.559599


In [42]:
from pathlib import Path

processed_path = Path("../data/processed/corn_exports_2025.csv")
weekly_totals.to_csv(processed_path, index=False)